In [1]:
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt

# function def.

In [455]:
def update_mutual_capacitance(Maxwell_cap_matrix,conductor1_index, conductor2_index, new_cap):
    """
    Args:
        Maxwell_cap_matrix: Maxwell capacitance matrix from Q3D
        conductor1_index : index of conductor 1
        conductor2_index : index of conductor 2
        new_cap : new capacitance for replacement
    """
    
    if conductor1_index == conductor2_index:
        raise('Error. First and second conductor index must be different!')
    
    M = np.copy(Maxwell_cap_matrix)
    
    M[conductor1_index, conductor2_index] = -new_cap
    M[conductor2_index, conductor1_index] = -new_cap
    
    for i in range(M.shape[0]):
        M[i,i] = -(np.sum(M[i,:]) - M[i,i])
        
    return M

def calculate_qubit_2bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_bus2 = index['bus2']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    # find beta_bus1, beta_bus2 and beta_readout
    #, where beta_bus1 = C_bus1/C_sigma 
    # and C_sigma = C_bus1 + C_bus2 + C_read + C_shunt

    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_bus2, index_bus2] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    b2 = np.linalg.solve(C_M_1, -C_M[:, index_bus2])
    beta_bus2 = b2[index_island1] - b2[index_island2]
    
    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
      
    # find C_bus1, C_bus2, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1, beta_bus1]
                  ,[beta_bus2, beta_bus2-1, beta_bus2, beta_bus2]
                ,[beta_read, beta_read, beta_read-1, beta_read]  
                ,[1, 1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    
    return c

def print_result_2bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_bus2 = {c[1]:.3f} fF\n' 
         + f'C_read = {c[2]:.3f} fF\n'
         + f'C_shunt = {c[3]:.3f} fF')

def calculate_qubit_1bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]

    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])

    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]

    Q = C_M.dot(V0)
    C_sigma = np.dot(Q, V0)
    print(f'C_sigma from charging energy = {C_sigma:.3f}')
    
    # find C_bus1, C_read and C_shunt
    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1]
                ,[beta_read,  beta_read-1, beta_read]  
                ,[1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    c = np.append(c, np.array([C_sigma, beta_bus1, beta_read]))
    
    return c

def print_result_1bus_1read(c):
    c = np.ndarray.flatten(c)
    print(f'C_bus1 = {c[0]:.3f} fF\n' 
         + f'C_read = {c[1]:.3f} fF\n'
         + f'C_shunt = {c[2]:.3f} fF\n'
         + f'C_sigma = {c[3]:.3f} fF\n'
         + f'beta_bus1 = {c[4]:.3f}\n'
         + f'beta_read = {c[5]:.3f}')

# 2-bus, 1-readout

In [64]:
C_M = np.array([[107.94,   -0.072728, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-0.072728, 107.72, -95.966, -0.72699, -9.5856, -0.041242]
               ,[-96.56,   -95.966, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -0.72699, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -9.5856,-55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219,-0.041242, -95.917,   -7.0491, -0.61872, 103.03]
              ]) 

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }

In [69]:
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result_2bus_1read)

C_bus1 = 4.248 fF
C_bus2 = 4.181 fF
C_read = -3.400 fF
C_shunt = 54.407 fF


In [74]:
C_M = update_mutual_capacitance(C_M,2,3,80)

In [78]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result_2bus_1read)
print(np.sum(result_2bus_1read))

C_bus1 = -4.929 fF
C_bus2 = -4.853 fF
C_read = 2.900 fF
C_shunt = 71.039 fF
64.15754886396333


In [79]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 4
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result_2bus_1read)
print(np.sum(result_2bus_1read))

C_bus1 = 4.929 fF
C_bus2 = 4.853 fF
C_read = -2.900 fF
C_shunt = 57.276 fF
64.1575488639633


# 1-bus, 1-readout

In [5]:
C_M = np.array([[107.94, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-96.56, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219, -95.917,   -7.0491, -0.61872, 103.03]
              ])

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [6]:
cap_result = calculate_qubit_1bus_1read_equiv_cap(C_M, conductor_index)
print_result(cap_result)

C_bus1 = -4.248 fF
C_read = 3.400 fF
C_shunt = 60.283 fF


In [7]:
C_M_1 = update_mutual_capacitance(C_M, 0, 3, 50)
cap_result1 = calculate_qubit_1bus_1read_equiv_cap(C_M_1, conductor_index)
print_result(cap_result1)

C_bus1 = -19.081 fF
C_read = 4.055 fF
C_shunt = 79.860 fF


In [8]:
7e-15/2*np.sqrt(6.26e9*5.05e9/55e-15/468e-15)/1e6

122.6582117411505

# ZZ gate, qubit 

In [9]:
C_M = np.array([[143.07500,-87.36528,-2.23553,-52.84648,-0.23655]
               ,[-87.36528,294.22820,-40.74111,-46.17334,-77.96845]
               ,[-2.23553,-40.74111,67.58779,-15.71175,-7.76007]
               ,[-52.84648,-46.17334,-15.71175,118.06270,-0.89528]
               ,[-0.23655,-77.96845,-7.76007,-0.89528,86.86034]
              ])
C_M_N4 = np.array([[105.30560,-79.80541,-1.58229,-23.78888,-0.09290]
                   ,[-79.80541,326.48530,-62.62674,-57.28538,-70.37290]
                   ,[-1.58229,-62.62674,100.41100,-26.05792,-8.15522]
                   ,[-23.78888,-57.28538,-26.05792,110.20110,-0.72336]
                   ,[-0.09290,-70.37290,-8.15522,-0.72336,79.40180]
                  ])

C_M_N5 = np.array([[119.07130,-84.60930,-2.05832,-32.15181,-0.11997]
                  ,[-84.60930,326.59040,-62.31408,-54.56391,-70.06300]
                  ,[-2.05832,-62.31408,100.36980,-25.74456,-8.16695]
                  ,[-32.15181,-54.56391,-25.74456,115.51110,-0.70659]
                  ,[-0.11997,-70.06300,-8.16695,-0.70659,79.16525]
                  ])

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 1
                   , 'island1' : 2
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'read' : 4
                  }

In [10]:
cap_result = calculate_qubit_1bus_1read_equiv_cap(C_M, conductor_index)
print_result(cap_result)

C_bus1 = -16.292 fF
C_read = 4.849 fF
C_shunt = 61.582 fF


In [11]:
cap_result_N4 = calculate_qubit_1bus_1read_equiv_cap(C_M_N4, conductor_index)
print_result(cap_result_N4)

C_bus1 = -10.320 fF
C_read = 3.990 fF
C_shunt = 71.860 fF


In [12]:
cap_result_N5 = calculate_qubit_1bus_1read_equiv_cap(C_M_N5, conductor_index)
print_result(cap_result_N5)

C_bus1 = -13.471 fF
C_read = 4.139 fF
C_shunt = 75.826 fF


In [13]:
import transmonCPW
from scipy.constants import h,e,k

In [14]:
Ec = e**2/2/75.82e-15/h/1e9
q = transmonCPW.Transmon.in_GHz(20, Ec)

In [15]:
Ec

0.2554765144691573

In [16]:
q.anharmonicity/1e6

-283.83999835346816

In [17]:
q.f01/1e6

6126.509351275784

In [18]:
q.Ej_to_Ec_ratio

78.28508245290986

In [19]:
q.charge_dispersion

2361.203378677368

In [255]:
def calculate_qubit_2bus_1read_equiv_cap(C_M, index):
    """
    Calculate effective shunt and coupling capacitance.
    Arg:
        C_M : Maxewll capacitance matrix
        index : index of each conductor in C_M, starting from 0
    Return:
        Tuple of effective bus, readout and shunt capacitance
    """
    index_bus1 = index['bus1']
    index_bus2 = index['bus2']
    index_gnd = index['gnd']
    index_island1 = index['island1']
    index_island2 = index['island2']
    index_read = index['read']
    
    NUM_CONDUCTOR = C_M.shape[0]
    
    # find beta_bus1, beta_bus2 and beta_readout
    #, where beta_bus1 = C_bus1/C_sigma 
    # and C_sigma = C_bus1 + C_bus2 + C_read + C_shunt

    C_M_1 = np.zeros((NUM_CONDUCTOR, NUM_CONDUCTOR))
    C_M_1[:, index_island1] = C_M[:, index_island1]
    C_M_1[:, index_island2] = C_M[:, index_island2]
    C_M_1[index_bus1, index_bus1] = -1
    C_M_1[index_bus2, index_bus2] = -1
    C_M_1[index_gnd, index_gnd] = -1
    C_M_1[index_read, index_read] = -1

    b1 = np.linalg.solve(C_M_1, -C_M[:, index_bus1])
    beta_bus1 = b1[index_island1] - b1[index_island2]
    
    b2 = np.linalg.solve(C_M_1, -C_M[:, index_bus2])
    beta_bus2 = b2[index_island1] - b2[index_island2]
    
    r = np.linalg.solve(C_M_1, -C_M[:, index_read])
    beta_read = r[index_island1] - r[index_island2]
    print(beta_bus1, beta_bus2, beta_read)
    
    # Find C_sigma
    # V_island1 - V_island2 = 1, Eq.1
    # Q_island1 + Q_island2 = 0, Eq.2
    # From Eq1 and Eq2, find V_island1 and V_island2 

    A = np.array([[1, -1],
                 [C_M[index_island1, index_island1]+C_M[index_island2, index_island1]
                  , C_M[index_island1, index_island2]+C_M[index_island2, index_island2]]])
    print(A)
    b = np.array([[1]
                  ,[0]])
    V = np.linalg.solve(A, b)  # A*V = b, solve for V[0]=V_island1, V[1]=V_island2

    V0 = np.zeros(NUM_CONDUCTOR)
    V0[index_island1] = V[0]
    V0[index_island2] = V[1]
    print(V0)
    
#     Q = C_M.dot(V0)
#     print(Q)
#     C_sigma = np.dot(Q, V0)
    summ = 0
    for i in range(6):
        for j in range(6):
            if i<j:
                summ = summ +  1/2*(-C_M[i,j])*(V0[i]-V0[j])**2
                    
    C_sigma = summ *2
    print(C_sigma)
    # find C_bus1, C_bus2, C_read and C_shunt
#     beta_bus1 = np.abs(beta_bus1)
#     beta_bus2 = np.abs(beta_bus2)
#     beta_read= np.abs(beta_read)

    A = np.array([[beta_bus1-1, beta_bus1, beta_bus1, beta_bus1]
                  ,[beta_bus2, beta_bus2-1, beta_bus2, beta_bus2]
                ,[beta_read, beta_read, beta_read-1, beta_read]  
                ,[1, 1, 1, 1]])
#     A = np.array([[beta_bus1-1, beta_bus1, 0, beta_bus1]
#               ,[beta_bus2, beta_bus2-1, 0, beta_bus2]
#             ,[0, 0, beta_read-1, beta_read]  
#             ,[1, 1, 1, 1]])
    b = np.array([[0]
                  ,[0]
                  ,[0]
                 ,[C_sigma]])
    c = np.linalg.solve(A, b)
    
    return c

In [256]:
C_M = np.array([[107.94,   -0.072728, -96.56, -0.72528, -9.7239,-0.041219]
               ,[-0.072728, 107.72, -95.966, -0.72699, -9.5856, -0.041242]
               ,[-96.56,   -95.966, 586.48,  -59.967, -55.133, -95.917]
               ,[-0.72528, -0.72699, -59.967, 92.027,  -23.543,   -7.0491]
               ,[-9.7239, -9.5856,-55.133, -23.543, 98.962,-0.61872]
               ,[-0.041219,-0.041242, -95.917,   -7.0491, -0.61872, 103.03]
              ]) 

# index dictionary of conductors in input Maxwell capacitance matrix
conductor_index = { 'gnd' : 2
                   , 'island1' : 3
                   , 'island2' : 4
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }

In [257]:
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M, conductor_index)
print_result_2bus_1read(result_2bus_1read)

-0.0714648191785757 -0.07034235776989464 0.0572043954280144
[[ 1.    -1.   ]
 [68.484 75.419]]
[ 0.          0.          0.          0.52409609 -0.47590391  0.        ]
59.34987196734467
C_bus1 = -4.241 fF
C_bus2 = -4.175 fF
C_read = 3.395 fF
C_shunt = 64.371 fF


In [246]:
C_M_1 = update_mutual_capacitance(C_M, 4,0, 30)


In [247]:
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M_1, conductor_index)
print_result_2bus_1read(result_2bus_1read)
np.sum(np.abs(result_2bus_1read))

-0.19114905630491585 -0.056529012954670554 0.06063900359136734
[[ 1.      -1.     ]
 [68.46837 95.33732]]
[ 0.         0.         0.         0.5820147 -0.4179853  0.       ]
[ 12.11743529   3.58352099 -11.8568923   63.39259802 -63.39259802
  -3.84406398]
63.39259802414923
C_bus1 = -12.117 fF
C_bus2 = -3.584 fF
C_read = 3.844 fF
C_shunt = 75.249 fF


94.79451059208995

In [248]:
conductor_index = { 'gnd' : 2
                   , 'island1' : 4
                   , 'island2' : 3
                   , 'bus1' : 0
                   , 'bus2' :1
                   , 'read' : 5
                  }
result_2bus_1read = calculate_qubit_2bus_1read_equiv_cap(C_M_1, conductor_index)
print_result_2bus_1read(result_2bus_1read)
np.sum(np.abs(result_2bus_1read))

0.19114905630491585 0.056529012954670554 -0.06063900359136734
[[ 1.      -1.     ]
 [95.33732 68.46837]]
[ 0.         0.         0.        -0.5820147  0.4179853  0.       ]
[-12.11743529  -3.58352099  11.8568923  -63.39259802  63.39259802
   3.84406398]
63.39259802414921
C_bus1 = 12.117 fF
C_bus2 = 3.584 fF
C_read = -3.844 fF
C_shunt = 51.536 fF


71.08072598265419

In [271]:
from scipy.constants import h,e
e**2/2/65.3e-15/h/1e9

0.29663444604979344

In [272]:
import transmonCPW

In [273]:
q = transmonCPW.Transmon.in_GHz(12,0.296)

In [274]:
q.f01/1e9

5.015187780303034

In [275]:
q.anharmonicity

-346879165.19573385

In [440]:
C_M_mod7 = np.array([[587.1,-39.97, -117.08,-40.814,-118.09]
               ,[-39.97,84.619,-12.5,-29.821,-1.1642]
               ,[-117.08,-12.5, 125.53,-1.1792,-0.13214]
               ,[-40.814,-29.821,-1.1792,85.265,-12.645]
               ,[-118.09,-1.1642,-0.13214,-12.645,125.61]
               ])

conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' : 3
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result_1bus_1read = calculate_qubit_1bus_1read_equiv_cap(C_M_mod7, conductor_index)
print_result_1bus_1read(result_1bus_1read)
np.sum(result_1bus_1read)


C_sigma from charging energy = 57.381
C_bus1 = -5.700 fF
C_read = 5.700 fF
C_shunt = 57.380 fF


57.38055363654504

In [453]:
e**2/2/(64.3)/1e-15/h/1e9

0.3012477344798057

In [293]:
import transmonCPW

In [449]:

q = transmonCPW.Transmon.in_GHz(13.4105, 0.323)

In [450]:
q.f01/1e9

5.5427039708357215

In [451]:
q.anharmonicity/1e6

-377.5862913052408

In [433]:
q.Ej_to_Ec_ratio


60.74074074074073

In [379]:
e**2/2/(69.99+4.82*2)/1e-15/h/1e9

0.24325291130291987

In [395]:
q = transmonCPW.Transmon.in_GHz(11.4, 0.2432)

In [396]:
q.f01/1e9

4.45167011802067

In [397]:
q.anharmonicity/1e9

-0.2809613403454797

In [398]:
62.4668*0.073

4.5600764

In [399]:
e**2/2/(62.4668+4.56*2)/1e-15/h/1e9
# e**2/2/(62.4668)/1e-15/h/1e9

0.2705838133154647

In [400]:
62.4668+4.56*2

71.5868

In [458]:
C_M_mod4 = np.array([[291.34,-51.173,-17.152,-58.758,-17.068]
               ,[-51.173,92.843,-8.6623,-30.712,-0.0641]
               ,[-17.152,-8.6623,25.913,-0.0779,-0.00136]
               ,[-58.758,-30.712,-0.07791,99.027,-8.6762]
               ,[-17.068,-0.0641,-0.001367,-8.6762, 25.913]
               ])

conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' :3 
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result_1bus_1read = calculate_qubit_1bus_1read_equiv_cap(C_M_mod4, conductor_index)
print_result_1bus_1read(result_1bus_1read)
np.sum(result_1bus_1read)


C_sigma from charging energy = 63.250
C_bus1 = -4.099 fF
C_read = 4.499 fF
C_shunt = 62.850 fF
C_sigma = 63.250 fF
beta_bus1 = -0.065
beta_read = 0.071


126.50675067218438

In [474]:
e**2/2/(78.25+7.8*2)/1e-15/h/1e9

0.20639562415611626

In [476]:
q = transmonCPW.Transmon.in_GHz(16.5, 0.206)
q.f01/1e9

4.999467350948322

In [477]:
q.anharmonicity/1e6

-228.54547331634254

In [439]:
C_M_mod7_1 = np.array([[568.7, -43.028,-118.33,-39.898,-117.51]
               ,[-43.028,87.197,-16.9,-28.218,-1.1171]
               ,[-118.33,-16.9, 133.82, -1.126,-0.110033]
               ,[-39.898, -28.218,-1.126, 87.699, -17.034]
               ,[-117.51,-1.1171, -0.11033, -17.034, 133.73]
               ])

conductor_index = { 'gnd' : 0
                   , 'island1' : 1
                   , 'island2' :3 
                   , 'bus1' : 4
                   , 'read' : 2
                  }
result_1bus_1read = calculate_qubit_1bus_1read_equiv_cap(C_M_mod7_1, conductor_index)
print_result_1bus_1read(result_1bus_1read)
np.sum(result_1bus_1read)


C_sigma from charging energy = 57.832
C_bus1 = -7.920 fF
C_read = 7.925 fF
C_shunt = 57.827 fF


57.8324681664697

In [438]:
57.827+7.92+7.925

73.672